# Cover Type Prediction using ensembles

## Dataset Description
The dataset represents the data about trees which were planted in the US. The dataset consists of the information about 500000 trees. Your aim is to build  Random Forest Ensemble to predict the cover type of trees. In order to successfully complete this assignment you have to follow this algorithm:
* Load the training data
* Transform categorical features into vector representations
* Split dataset into the train and validation part
* Fit the Random Forest Ensemble into the training set
* Compare the accuracy of the fitted model with the Logistic Regression Model, which is about 0.67 for this set


If you have enough time, it will be very interesting to dig into further research through these steps:
* Determine which features are valuable for your model (calculate feature importance of your model).
* Try to reduce number of trees and see the results.
* Understand why the linear models have poor performance on this dataset.


## Loading data

Init pyspark session

In [1]:
from __future__ import division, print_function, unicode_literals # For the compatibility with Python 2

In [2]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
                            .enableHiveSupport()\
                            .appName("spark_ml")\
                            .master("local[4]")\
                            .getOrCreate()

Load train dataset located at /data/covertype2 with at least 60 partitions (use function repartition for this case). Use option `inferSchema` to save numerical features.

In [7]:
df = spark_session.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferschema", "true")\
        .option("mode", "DROPMALFORMED")\
        .load("/data/covertype2/train.csv")\
        .repartition(60)

## Transforming data

As you can see, there are two categorical features in dataset: 'Soil_Type' and 'Wild_Type'. You have to transform them into the vector embeddings.

First of all, you have to use StringIndexer to transform feature types to indexes

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
cat_cols=['Soil_Type','Wild_Type']
cat_cols_index={'Soil_Type':'Soil_Index','Wild_Type':'Wild_Index'}
cat_cols_encoder={'Soil_Index':'SoilEncoder','Wild_Index':'WildEncoder'}


In [16]:
cat_indexers = [ StringIndexer(inputCol=col, outputCol=cat_cols_index[col])
                 for col in cat_cols ]

Apply OneHotEncoder technique to the dataset in order to get vectors for the Random Forest classification

In [17]:
from pyspark.ml.feature import OneHotEncoder

In [18]:
encoders = [ OneHotEncoder(dropLast=False, inputCol=indexer.getOutputCol(),
                           outputCol=cat_cols_encoder[indexer.getOutputCol()])
             for indexer in cat_indexers ]


Use the VectorAssembler technique to accumulate all features into one vector. Don't forget to use features that you have generated

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
vector_assembler = VectorAssembler(inputCols=['SoilEncoder', # feature name of Soil type encoded
                                              'WildEncoder', # feature name of Wild type encoded
                                              'Elevation',
                                              'Aspect',
                                              'Slope',
                                              'Horizontal_Distance_To_Hydrology',
                                              'Vertical_Distance_To_Hydrology',
                                              'Horizontal_Distance_To_Roadways',
                                              'Hillshade_9am',
                                              'Hillshade_Noon',
                                              'Hillshade_3pm',
                                              'Horizontal_Distance_To_Fire_Points'
                                              ], outputCol='features')

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

## Training

Fit the Random Forest model to the train dataset. Don't forget to split dataset into two parts to check your trained models. It is desirable to use about 100 trees with depth about 7 in order to avoid wasting too much time waiting while your model will be fit to the data. Try to adjust the options 'subsamplingRate' and 'featureSubsetStrategy' to get better results

<b> Extra task.</b> Use the Cross-Validation to check your model.

In [25]:
rf = RandomForestClassifier(labelCol='Target',featuresCol='features', numTrees=100,maxDepth=7)

In [26]:
pipeline = Pipeline(stages=[ *cat_indexers, *encoders, vector_assembler, rf ] )

In [27]:
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed = 23)
model = pipeline.fit(trainingData)

Get the feature importances of the trained model. What 5 features are the most important in the dataset?

In [31]:
#model.stages[5].featureImportances

SparseVector(54, {0: 0.0066, 1: 0.0178, 2: 0.0031, 3: 0.0012, 4: 0.0471, 5: 0.0417, 6: 0.0029, 7: 0.0381, 8: 0.0007, 9: 0.0006, 10: 0.005, 11: 0.0311, 12: 0.026, 13: 0.0026, 14: 0.0317, 15: 0.0003, 16: 0.012, 17: 0.0194, 18: 0.0067, 19: 0.0031, 20: 0.0001, 21: 0.0007, 22: 0.0025, 23: 0.0001, 24: 0.0, 25: 0.0, 26: 0.0006, 27: 0.0001, 28: 0.0001, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0001, 33: 0.0001, 34: 0.0001, 35: 0.0001, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0215, 41: 0.0238, 42: 0.1467, 43: 0.0114, 44: 0.3888, 45: 0.0067, 46: 0.0069, 47: 0.0116, 48: 0.0065, 49: 0.0314, 50: 0.006, 51: 0.0103, 52: 0.0043, 53: 0.0221})

Apply model to the validation part of your set and get the accuracy score for the data. Use the MulticlassClassificationEvaluator class from the ml.evaluation module. 

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [35]:
predictions = model.transform(testData)
evaluator = MulticlassClassificationEvaluator(labelCol = "Target", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)

subsamplingRate=1.0

0.6960731518180252


Are your results better than the results from the Logistic Regression model?

# Performing test submission

Apply the models to the test dataset.

<b>Note!</b> Dataset will be changed during the test phase. Your last cell output must be the accuracy score.

In [41]:
# Load dataset, transform dataset
dfTest = spark_session.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferschema", "true")\
        .option("mode", "DROPMALFORMED")\
        .load("/data/covertype2/test.csv")\
        .repartition(60)
dfTest.printSchema()
dfTestfinal=model.transform(dfTest)

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wild_Type: string (nullable = true)
 |-- Soil_Type: string (nullable = true)
 |-- Target: integer (nullable = true)



In [43]:
# Calculate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = "Target", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(dfTestfinal)
spark_session.stop()


0.6980838675121378


In [45]:
# This is last cell. The results of the evaluator must be here
print(accuracy*100)

69.80838675121377
